In [1]:
# imports
import math
from PIL import Image, ExifTags
import piexif

# Ensure pillow-heif is available and registered for HEIC support
try:
    import pillow_heif
    pillow_heif.register_heif_opener()
except ModuleNotFoundError:
    import sys, subprocess
    print("Installing pillow-heif... this may take a minute")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow-heif"])  # installs in the current kernel
    import pillow_heif
    pillow_heif.register_heif_opener()

In [2]:
# Convert image from .HEIC to jpg
# Step 1: Open HEIC and extract metadata
heif_file = pillow_heif.open_heif("mouse.HEIC", convert_hdr_to_8bit=False, bgr_mode=False)
image = heif_file[0].to_pillow()
exif_data = heif_file[0].info.get("exif", None)

# Step 2: Save image as JPEG
image.save("mouse.jpg", format="JPEG")

# Step 3: Write metadata to JPEG (if available)
if exif_data:
    piexif.insert(exif_data, "mouse.jpg")

In [3]:
# print meta data
from PIL.ExifTags import TAGS

def print_exif(image_path):
    with Image.open(image_path) as img:
        exif_data = img._getexif()
        if exif_data is None:
            print("No EXIF data found.")
            return
        for tag_id, value in exif_data.items():
            tag = TAGS.get(tag_id, tag_id)
            print(f"{tag}: {value}")

if __name__ == "__main__":
    image_path = "mouse.jpg"  # Change to your image file path
    print_exif(image_path)

GPSInfo: {1: 'N', 2: (13.0, 53.0, 49.9), 3: 'E', 4: (100.0, 38.0, 27.57), 5: b'\x00', 6: 3.811237967315872, 7: (8.0, 18.0, 31.0), 12: 'K', 13: 0.07194754482146858, 16: 'M', 17: 167.4727171492205, 23: 'M', 24: 167.4727171492205, 29: '2025:10:13', 31: 17.110775692348078}
ResolutionUnit: 2
ExifOffset: 216
Make: Apple
Model: iPhone 16
Software: 18.6.2
Orientation: 6
DateTime: 2025:10:13 15:18:31
XResolution: 72.0
YResolution: 72.0
HostComputer: iPhone 16
ExifVersion: b'0232'
ShutterSpeedValue: 5.643639798488665
ApertureValue: 1.3561438532196026
DateTimeOriginal: 2025:10:13 15:18:31
DateTimeDigitized: 2025:10:13 15:18:31
BrightnessValue: 1.3332337692816454
ExposureBiasValue: 0.0
MeteringMode: 5
ColorSpace: 65535
Flash: 16
FocalLength: 5.960000038146973
ExifImageWidth: 5712
ExifImageHeight: 4284
FocalLengthIn35mmFilm: 26
OffsetTime: +07:00
OffsetTimeOriginal: +07:00
OffsetTimeDigitized: +07:00
SubsecTimeOriginal: 610
SubjectLocation: (2855, 2139, 3291, 1880)
SubsecTimeDigitized: 610
SensingM

In [4]:
def _rational_to_float(v):
    if isinstance(v, tuple) and len(v) == 2 and v[1]:
        return float(v[0]) / float(v[1])
    try:
        return float(v)
    except Exception:
        return None

def get_exif_info(image_path):
    img = Image.open(image_path)
    exif_bytes = img.info.get("exif", None)
    if not exif_bytes:
        return None
    exif_dict = piexif.load(exif_bytes)

    focal_length = _rational_to_float(exif_dict["Exif"].get(piexif.ExifIFD.FocalLength))
    focal_35 = exif_dict["Exif"].get(piexif.ExifIFD.FocalLengthIn35mmFilm)
    focal_35 = int(focal_35) if focal_35 else None

    # Pixel pitch from FocalPlaneX/YResolution (if present)
    xres = _rational_to_float(exif_dict["Exif"].get(piexif.ExifIFD.FocalPlaneXResolution))
    yres = _rational_to_float(exif_dict["Exif"].get(piexif.ExifIFD.FocalPlaneYResolution))
    unit = exif_dict["Exif"].get(piexif.ExifIFD.FocalPlaneResolutionUnit)  # 2=inches, 3=cm
    mm_per_unit = {2: 25.4, 3: 10.0, 4: 1.0, 5: 0.001}.get(unit, None)
    pixel_pitch_mm = (mm_per_unit / xres) if (xres and mm_per_unit) else None

    # Digital zoom (if present)
    dz = _rational_to_float(exif_dict["Exif"].get(piexif.ExifIFD.DigitalZoomRatio))

    model = exif_dict["0th"].get(piexif.ImageIFD.Model, b"").decode(errors="ignore")
    make = exif_dict["0th"].get(piexif.ImageIFD.Make, b"").decode(errors="ignore")

    width_px, height_px = img.size
    return {
        "focal_length_mm": focal_length,
        "focal_length_35mm": focal_35,
        "pixel_pitch_mm": pixel_pitch_mm,   # None if not in EXIF
        "digital_zoom": dz,
        "model": model,
        "make": make,
        "image_width_px": width_px,
        "image_height_px": height_px,
    }

def estimate_sensor_width(focal_length_mm, focal_length_35mm):
    """
    Estimate sensor width (mm) using crop factor if actual sensor size is unknown.
    - crop_factor = focal_35mm / focal_mm
    - sensor_width = 36 mm / crop_factor   (based on full-frame width = 36 mm)
    """
    if not focal_length_mm or not focal_length_35mm:
        return None
    crop_factor = focal_length_35mm / focal_length_mm
    return 36.0 / crop_factor
    
def compute_object_length(pixel_length, image_height_px, focal_length_mm, sensor_width_mm, distance_mm):
    """
    Compute real-world object length in mm.
    Formula:
    - hfov = 2 * atan(sensor_width / (2 * focal_length))
    - physical_width = 2 * distance * tan(hfov/2)
    - mm_per_pixel = physical_width / image_width_px
    - object_length_mm = pixel_length * mm_per_pixel
    """
    if not all([pixel_length, image_height_px, focal_length_mm, sensor_width_mm, distance_mm]):
        raise ValueError("Missing required parameters.")

    # hfov = 2.0 * math.atan(sensor_width_mm / (2.0 * focal_length_mm))
    # physical_width_mm = 2.0 * distance_mm * math.tan(hfov / 2.0)
    # mm_per_pixel = physical_width_mm / image_width_px
    # return pixel_length * mm_per_pixel
    pixel_pitch_mm = sensor_width_mm / image_height_px
    size_image_mm = pixel_length * pixel_pitch_mm
    actual_length_mm = (size_image_mm * distance_mm) / focal_length_mm
    return actual_length_mm

# Path to your photo
image_path = "mouse.jpg"

# Step 1: Extract EXIF
exif_info = get_exif_info(image_path)
print("EXIF info:", exif_info)

# Step 2: Get or estimate sensor width
sensor_width_mm = 5.96  # for iphone 16
if not sensor_width_mm and exif_info["focal_length_mm"] and exif_info["focal_length_35mm"]:
    sensor_width_mm = estimate_sensor_width(exif_info["focal_length_mm"], exif_info["focal_length_35mm"])
    print("Estimated sensor width:", sensor_width_mm)

# Step 3: Known measurement inputs
distance_mm = 380       # distance camera -> object in mm (must measure or estimate) Estimate
pixel_length = 1886.9374658424692  # object's pixel length (e.g., contour bounding box width)

# Step 4: Compute real-world length
real_length_mm = compute_object_length(
    pixel_length=pixel_length,
    image_height_px=exif_info["image_height_px"],
    focal_length_mm=exif_info["focal_length_mm"],
    sensor_width_mm=sensor_width_mm,
    distance_mm=distance_mm
)

print(f"Object length ≈ {real_length_mm:.2f} mm ({real_length_mm/10:.2f} cm)")
print(f"Ground Truth Length: 125 mm, 12.5 cm")

EXIF info: {'focal_length_mm': 5.960000038146973, 'focal_length_35mm': 26, 'pixel_pitch_mm': None, 'digital_zoom': None, 'model': 'iPhone 16', 'make': 'Apple', 'image_width_px': 4284, 'image_height_px': 5712}
Object length ≈ 125.53 mm (12.55 cm)
Ground Truth Length: 125 mm, 12.5 cm
